## Data Analysis and Visualization

### Use SQL

In [51]:
!pip install sqlalchemy==1.3.9

  Using cached SQLAlchemy-1.3.9-cp311-cp311-macosx_11_0_arm64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.25
    Uninstalling SQLAlchemy-2.0.25:
      Successfully uninstalled SQLAlchemy-2.0.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.1 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
langchain-community 0.0.13 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.


In [52]:
!pip3 install ipython-sql

  Obtaining dependency information for sqlalchemy>=2.0 from https://files.pythonhosted.org/packages/3f/ec/d285f944d47d885a84fdd2c2c7b1de10a106a7206a7d56325b772d09ffeb/SQLAlchemy-2.0.25-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached SQLAlchemy-2.0.25-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.6 kB)
Using cached SQLAlchemy-2.0.25-cp311-cp311-macosx_11_0_arm64.whl (2.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [53]:
# Connect to the data base 
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [54]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

!pip install -q pandas==1.1.5

%sql sqlite:///my_data1.db

In [55]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [56]:
# used to remove blank rows from the table
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null


 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [57]:
# Display the names of the unique launch sites in the space mission 
%sql select distinct(LAUNCH_SITE) from SPACEXTBL


 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [58]:
# Display 5 records where launch sites begin with the string 'CCA'
%sql select * from SPACEXTBL where LAUNCH_SITE like 'CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [59]:
# Display the total payload mass carried by boosters launched by NASA (CRS)
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


In [60]:
# Display average payload mass carried by booster version F9 v1.1
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2928.4


In [61]:
# List the date when the first succesful landing outcome in ground pad was acheived.
%sql select min(Date) from SPACEXTBL where Landing_Outcome = 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


min(Date)
2015-12-22


In [62]:
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql select Booster_Version from SPACEXTBL where Landing_Outcome = 'Success (drone ship)' and PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [63]:
# List the total number of successful and failure mission outcomes
%sql SELECT count(Mission_Outcome) from SPACEXTBL where Mission_Outcome = 'Success' or Mission_Outcome = 'Failure';

 * sqlite:///my_data1.db
Done.


count(Mission_Outcome)
98


In [64]:
# List the names of the booster_versions which have carried the maximum payload mass. Use a subquery
%sql SELECT Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [65]:
# List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
%sql SELECT substr(Date, 6,2) AS month_name, landing_outcome AS failure_landing_outcomes, booster_version,launch_site from SPACEXTBL where substr(Date,0,5)='2015'

 * sqlite:///my_data1.db
Done.


month_name,failure_landing_outcomes,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
02,Controlled (ocean),F9 v1.1 B1013,CCAFS LC-40
03,No attempt,F9 v1.1 B1014,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
04,No attempt,F9 v1.1 B1016,CCAFS LC-40
06,Precluded (drone ship),F9 v1.1 B1018,CCAFS LC-40
12,Success (ground pad),F9 FT B1019,CCAFS LC-40


In [66]:
# Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.
%sql select * from SPACEXTBL where Landing_Outcome like 'Success%' and (Date between '2010-06-04' and '2017-03-20') order by date desc

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2017-02-19,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
2017-01-14,17:54:00,F9 FT B1029.1,VAFB SLC-4E,Iridium NEXT 1,9600,Polar LEO,Iridium Communications,Success,Success (drone ship)
2016-08-14,5:26:00,F9 FT B1026,CCAFS LC-40,JCSAT-16,4600,GTO,SKY Perfect JSAT Group,Success,Success (drone ship)
2016-07-18,4:45:00,F9 FT B1025.1,CCAFS LC-40,SpaceX CRS-9,2257,LEO (ISS),NASA (CRS),Success,Success (ground pad)
2016-05-27,21:39:00,F9 FT B1023.1,CCAFS LC-40,Thaicom 8,3100,GTO,Thaicom,Success,Success (drone ship)
2016-05-06,5:21:00,F9 FT B1022,CCAFS LC-40,JCSAT-14,4696,GTO,SKY Perfect JSAT Group,Success,Success (drone ship)
2016-04-08,20:43:00,F9 FT B1021.1,CCAFS LC-40,SpaceX CRS-8,3136,LEO (ISS),NASA (CRS),Success,Success (drone ship)
2015-12-22,1:29:00,F9 FT B1019,CCAFS LC-40,OG2 Mission 2 11 Orbcomm-OG2 satellites,2034,LEO,Orbcomm,Success,Success (ground pad)


### Data Visualization

**Objectives**
* Perform exploratory Data Analysis and Feature Engineering using Pandas and Matplotlib

* Exploratory Data Analysis
* Preparing Data Feature Engineering